In [ ]:
import sys
sys.path.append('..')
print(sys.path)

In [1]:
from sys import exit
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from bulletin.commom import static, normalize

from bulletin.commom.utils import Timer, auto_fit_columns
pd.set_option('display.max_columns', None)

meses = ['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro']
pd.set_option("display.precision", 5)

In [2]:
csv_geral = pd.read_csv('geral.csv', sep=';', converters={'IDADE_ORIGINAL': lambda x: normalize.normalize_number(x,fill=0)}, parse_dates=['DATA_DIAGNOSTICO','DATA_OBITO'], date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
csv_geral.columns = [ normalize.normalize_labels(x) for x in csv_geral.columns ]
casos = csv_geral.copy()

In [3]:
faixa_etaria = [0,60,70,80,90,100,np.inf]
faixa_etaria_labels = ['<=59','60-69','70-79','80-89','90-99','>=100']

In [4]:
casos['mes_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%m'), axis=1)
casos['ano_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%Y'), axis=1)
casos['faixa_etaria'] = pd.cut(casos['idade_original'], faixa_etaria, labels=faixa_etaria_labels)

In [5]:
obitos = casos.loc[ casos['data_obito'].notnull() ].copy()
obitos['mes_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%m'), axis=1)
obitos['ano_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%Y'), axis=1)
obitos['faixa_etaria'] = pd.cut(obitos['idade_original'], faixa_etaria, labels=faixa_etaria_labels)

In [6]:
faixa_etaria_casos = casos.groupby(by=['ano_caso','mes_caso','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
faixa_etaria_casos = faixa_etaria_casos.unstack().fillna(0).astype(int)
faixa_etaria_casos = faixa_etaria_casos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

faixa_etaria_casos.columns = pd.MultiIndex.from_tuples(faixa_etaria_casos.columns)
faixa_etaria_casos['total'] = faixa_etaria_casos.xs('qtde',level=1, axis=1).sum(axis=1)


# for ano in faixa_etaria_casos.index.get_level_values(0).unique():
#     faixa_etaria_casos = faixa_etaria_casos.append(
#         pd.Series(data=faixa_etaria_casos.loc[ano].sum(0), name=(ano,'total'))
#     )

# faixa_etaria_casos = faixa_etaria_casos.append(
#         pd.Series(data=faixa_etaria_casos.xs('total', level=1).sum(0), name=('total_geral',''))
# )

faixa_etaria_casos = faixa_etaria_casos.loc[faixa_etaria_casos.sum(1)>0]
faixa_etaria_casos = faixa_etaria_casos.sort_index()

for ano_mes in faixa_etaria_casos.index:
    for faixa_etaria, tipo in faixa_etaria_casos.columns:
        if tipo == '%':
            faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_casos.loc[ano_mes,'total'].values
            
faixa_etaria_casos

<=59           60-69          70-79          80-89  \
                     qtde        %   qtde        %  qtde        %  qtde   
ano_caso mes_caso                                                         
2020     03           313  0.84367     44  0.11860    12  0.03235     2   
         04          1642  0.82929    165  0.08333   100  0.05051    61   
         05          4644  0.88491    338  0.06441   184  0.03506    74   
         06         24549  0.88580   1861  0.06715   890  0.03211   365   
         07         59009  0.87916   4774  0.07113  2239  0.03336   922   
         08         59211  0.88025   4715  0.07009  2234  0.03321   922   
         09         50052  0.87963   4076  0.07163  1900  0.03339   722   
         10         33925  0.87832   2811  0.07278  1276  0.03304   531   
         11         84805  0.88655   6817  0.07127  2898  0.03030   980   
         12        111431  0.86838  10288  0.08017  4666  0.03636  1664   
2021     01         98367  0.86239   9364  0.08209  4453  0.03904  1626   
         02         84437  0.86071   8229  0.08388  3886  0.03961  1357   
         03         60322  0.85709   6055  0.08603  2864  0.04069   970   

                           90-99          >=100            total  
                         %  qtde        %  qtde        %          
ano_caso mes_caso                                                 
2020     03        0.00539     0  0.00000     0  0.00000     371  
         04        0.03081    12  0.00606     0  0.00000    1980  
         05        0.01410     8  0.00152     0  0.00000    5248  
         06        0.01317    47  0.00170     2  0.00007   27714  
         07        0.01374   170  0.00253     6  0.00009   67120  
         08        0.01371   179  0.00266     5  0.00007   67266  
         09        0.01269   146  0.00257     5  0.00009   56901  
         10        0.01375    79  0.00205     3  0.00008   38625  
         11        0.01024   152  0.00159     5  0.00005   95657  
         12        0.01297   253  0.00197    19  0.00015  128321  
2021     01        0.01426   246  0.00216     7  0.00006  114063  
         02        0.01383   183  0.00187     9  0.00009   98101  
         03        0.01378   165  0.00234     4  0.00006   70380

In [7]:
faixa_etaria_obitos = obitos.groupby(by=['ano_obito','mes_obito','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
faixa_etaria_obitos = faixa_etaria_obitos.unstack().fillna(0).astype(int)
faixa_etaria_obitos = faixa_etaria_obitos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

faixa_etaria_obitos.columns = pd.MultiIndex.from_tuples(faixa_etaria_obitos.columns)
faixa_etaria_obitos['total'] = faixa_etaria_obitos.xs('qtde',level=1, axis=1).sum(axis=1)


# for ano in faixa_etaria_obitos.index.get_level_values(0).unique():
#     faixa_etaria_obitos = faixa_etaria_obitos.append(
#         pd.Series(data=faixa_etaria_obitos.loc[ano].sum(0), name=(ano,'total'))
#     )

# faixa_etaria_obitos = faixa_etaria_obitos.append(
#         pd.Series(data=faixa_etaria_obitos.xs('total', level=1).sum(0), name=('total_geral',''))
# )

faixa_etaria_obitos = faixa_etaria_obitos.loc[faixa_etaria_obitos.sum(1)>0]
faixa_etaria_obitos = faixa_etaria_obitos.sort_index()

for ano_mes in faixa_etaria_obitos.index:
    for faixa_etaria, tipo in faixa_etaria_obitos.columns:
        if tipo == '%':
            faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_obitos.loc[ano_mes,'total'].values
            
faixa_etaria_obitos

<=59          60-69          70-79          80-89  \
                    qtde        %  qtde        %  qtde        %  qtde   
ano_obito mes_obito                                                     
2020      03           2  0.40000     1  0.20000     1  0.20000     1   
          04          33  0.32039    16  0.15534    22  0.21359    21   
          05          35  0.28455    34  0.27642    29  0.23577    22   
          06         161  0.26179   137  0.22276   158  0.25691   141   
          07         409  0.28403   335  0.23264   378  0.26250   256   
          08         411  0.26178   398  0.25350   408  0.25987   287   
          09         303  0.24318   311  0.24960   338  0.27127   222   
          10         180  0.22250   214  0.26452   217  0.26823   163   
          11         232  0.23062   265  0.26342   270  0.26839   201   
          12         507  0.24317   531  0.25468   581  0.27866   386   
2021      01         387  0.22592   464  0.27087   469  0.27379   323   
          02         436  0.26798   423  0.25999   444  0.27289   266   
          03         615  0.33154   476  0.25660   444  0.23935   266   

                             90-99          >=100          total  
                           %  qtde        %  qtde        %        
ano_obito mes_obito                                               
2020      03         0.20000     0  0.00000     0  0.00000     5  
          04         0.20388    11  0.10680     0  0.00000   103  
          05         0.17886     3  0.02439     0  0.00000   123  
          06         0.22927    18  0.02927     0  0.00000   615  
          07         0.17778    59  0.04097     3  0.00208  1440  
          08         0.18280    64  0.04076     2  0.00127  1570  
          09         0.17817    69  0.05538     3  0.00241  1246  
          10         0.20148    35  0.04326     0  0.00000   809  
          11         0.19980    37  0.03678     1  0.00099  1006  
          12         0.18513    75  0.03597     5  0.00240  2085  
2021      01         0.18856    68  0.03970     2  0.00117  1713  
          02         0.16349    54  0.03319     4  0.00246  1627  
          03         0.14340    52  0.02803     2  0.00108  1855

In [8]:
def set_format_cols(worksheet,cols,format):
    for col in cols:
        worksheet.set_column(f"{col}:{col}", None, format)


writer = pd.ExcelWriter("faixa_etaria_por_mes.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

workbook = writer.book

format1 = workbook.add_format({'num_format': '#.##0'})
# format2 = workbook.add_format({'num_format': '0,00%'})

faixa_etaria_casos.to_excel(writer,sheet_name=f"faixa_etaria_casos")
worksheet = writer.sheets[f"faixa_etaria_casos"]
# set_format_cols(worksheet,['C','E','G','I','K','M','O'],format1)
# set_format_cols(worksheet,['D','F','H','J','L','N'],format2)
auto_fit_columns(worksheet,faixa_etaria_casos)


faixa_etaria_obitos.to_excel(writer,sheet_name=f"faixa_etaria_obitos")
worksheet = writer.sheets[f"faixa_etaria_obitos"]
# set_format_cols(worksheet,['C','E','G','I','K','M','O'],format1)
# set_format_cols(worksheet,['D','F','H','J','L','N'],format2)
auto_fit_columns(worksheet,faixa_etaria_obitos)

writer.save()
writer.close()